# Import

In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')
#Content Based Recommender
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

# Read from file

In [13]:
df = pd.read_csv('cars500.csv')
df.head(5)

,city,brand,name,year,price,body,capacity,capacity_type,mileage,transmission,wheel,color,drive_unit,customs,image,description,avg_price,view_amount
0,Алматы,Carina ED,Toyota,1995,1350000,седан,2.0,petrol,NaN,автомат,справа,черный,передний привод,Да,https://photos-b-kl.kcdn.kz/1d/1d210e94-2486-4...,NaN,1267000,0
1,Астана,A8,Audi,1995,1600000,седан,3.7,petrol,264037.0,типтроник,слева,серебристый,передний привод,Да,https://photos-a-kl.kcdn.kz/68/6802c05b-8272-4...,"литые диски, тонировка, люк, противотуманки, к...",1651000,0
2,Алматы,Escalade,Cadillac,2004,5400000,внедорожник,6.0,petrol,NaN,автомат,слева,NaN,NaN,Да,https://photos-b-kl.kcdn.kz/0c/0ccc7792-c563-4...,NaN,5700000,0
3,Алматы,Continental GT,Bentley,2005,15000000,купе,6.0,petrol,NaN,автомат,слева,NaN,NaN,Да,https://photos-b-kl.kcdn.kz/6e/6e49b0fc-f8c0-4...,NaN,12750000,0
4,Алматы,Windom,Toyota,1997,1600000,седан,2.5,petrol,NaN,автомат,справа,NaN,передний привод,Да,https://photos-a-kl.kcdn.kz/a5/a5ca70fe-e7cf-4...,"литые диски, тонировка, аудиосистема, встроенн...",1885000,0


# Sets for null vals in description "" 

In [5]:
df.description = df.description.fillna("")

# Term Frequency Inverse Document Frequency

In [6]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(df.description)

# linear_kernel matrix distance

In [7]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim1 = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Recomendation fields

In [8]:
smd = df.reset_index()
titles = df["name"]+" "+df["brand"]
indices = pd.Series(df.index, index=df["name"]+" "+df["brand"]).drop_duplicates()
indices.shape

(11668,)

# Top 10 Recomendations function

In [9]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    print(sim_scores)
    sim_scores = sorted(sim_scores, reverse=True)
    sim_scores = sim_scores[1:31]
    m_indices = [i[0] for i in sim_scores]
    ttt = ", ".join(titles.iloc[m_indices].head(10))
    return ttt

# Bots code 

In [1]:
import telebot
from telebot import types

bot = telebot.TeleBot("")

@bot.message_handler(commands=['start'])
def command_start(message):
    bot.send_message(message.chat.id,
                     "Hello, I'm the demo ML bot."
                     " I can Predict you."
                     " Use /buy to order one, /perdict for Terms and Conditions")


@bot.message_handler(commands=['perdict'])
def command_pay(message):
    bot.send_message(message.chat.id,"Write name", parse_mode='Markdown')

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    tt = get_recommendations(message.text)
    bot.reply_to(message, tt)


bot.polling()